In [ ]:
### Collect GDP data from the FRED API ###

# Install the required package if not already installed
#pip install fredapi
#pip install pandas pandas_datareader

"""
To use FRED API, you need to register for an API key at: https://fred.stlouisfed.org/
API documentation can be found at: https://fred.stlouisfed.org/docs/api/fred/
"""

# Import necessary libraries
import os
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime
import yfinance as yf

# FRED API: Yon need to register for an API key at https://fred.stlouisfed.org/
os.environ["FRED_API_KEY"] = "ADD ON YOUR FRED API KEY HERE" 

In [ ]:
# Define the real GDP series IDs and their start dates for each country
# Note: The series IDs are based on the FRED database and may change over time.
gdp_series = {
    'United States': ('GDPC1', datetime(1992, 1, 1)),           # https://fred.stlouisfed.org/series/GDPC1 (Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate)
    'United Kingdom': ('NGDPRSAXDCGBQ', datetime(1991, 1, 1)),  # https://fred.stlouisfed.org/series/NGDPRSAXDCGBQ (Millions of Domestic Currency, Seasonally Adjusted)
    'Germany': ('CLVMNACSCAB1GQDE', datetime(1991, 1, 1)),      # https://fred.stlouisfed.org/series/CLVMNACSCAB1GQDE (Millions of Chained 2010 Euros, Seasonally Adjusted)
    #'South Korea': ('NGDPRSAXDCKRQ', datetime(1991, 1, 1)),    # https://fred.stlouisfed.org/series/NGDPRSAXDCKRQ (Millions of Domestic Currency, Seasonally Adjusted)
    'China': ('CHNGDPNQDSMEI', datetime(1992, 1, 1)),           # https://fred.stlouisfed.org/series/CHNGDPNQDSMEI (Yuan Renminbi, Seasonally Adjusted)
    #'Brazil': ('NGDPRSAXDCBRQ', datetime(1996, 1, 1)),         # https://fred.stlouisfed.org/series/NGDPRSAXDCBRQ (Millions of Domestic Currency, Seasonally Adjusted)
    #'India': ('NAEXKP01INQ652S', datetime(1994, 4, 1)),        # https://fred.stlouisfed.org/series/RGDPNAINA666NRUG (Indian Rupee, Seasonally Adjusted)
}

end_date = datetime(2024, 12, 31)

# Fetch and save GDP data for each country
for country, (series_id, start_date) in gdp_series.items():
    try:
        data = web.DataReader(series_id, 'fred', start_date, end_date)
        data = data.rename(columns={series_id: 'Real GDP'})
        data.to_csv(f"quarterly_rgdp_{country.replace(' ', '_')}.csv")
        print(f"{country}: {len(data)} rows saved.")
    except Exception as e:
        print(f"Error loading {country}: {e}")

United States: 132 rows saved.
United Kingdom: 136 rows saved.
Germany: 136 rows saved.
South Korea: 136 rows saved.
China: 127 rows saved.
Brazil: 116 rows saved.
India: 110 rows saved.


In [ ]:
## Collect US macroeconomic data from FRED API
"""
Note that S&P 500 Index or Wilshire 5000 Total Market Index are not available in FRED now.
Instead, we collect S&P 500 index from Yahoo Finance in below cell.
"""
us_monthly_series = {
    'working_hour_manuf': 'AWHMAN',        # Avg_Weekly_Hours_Manufacturing
    'CPI': 'CPIAUCSL',                     # CPI_All_Items
    'Ids_Prd': 'INDPRO',                   # Industrial_Production
    'R_Csump': 'DPCERA3M086SBEA',          # Real_Personal_Consumption_Expenditures
    'Ef_Fed_Rate': 'FEDFUNDS',             # Effective_Fed_Funds_Rate'
    'Intr_10Y': 'GS10',                    # Interest_Rate_10Y
    'Unemp': 'UNRATE',                     # Unemployment_Rate
    'M2': 'M2SL',                          # M2_Money_Stock
    'M1': 'M1SL',                          # M1_Money_Stock
    'Labor_partic': 'CIVPART',             # Civilian_Labor_Force_Participation_Rate
    'Per_savings': 'PSAVERT',              # Personal_Savings_Rate
    'All_Emp': 'PAYEMS',                   # All_Employees_Total_Nonfarm
    'Moody_aaa': 'AAA',                    # Moody's_Aaa_Corporate_Bond_Yield_Spread
    'NetExports': 'BOPTEXP',               # Net_Exports_of_Goods_and_Services
}

start_date = datetime(1992, 1, 1)
end_date = datetime(2024, 12, 31)

monthly_data = pd.DataFrame()

for name, series_id in us_monthly_series.items():
    try:
        series = web.DataReader(series_id, 'fred', start_date, end_date)
        series = series.rename(columns={series_id: name})
        if monthly_data.empty:
            monthly_data = series
        else:
            monthly_data = monthly_data.join(series, how='outer')
        print(f"{name} loaded.")
    except Exception as e:
        print(f"Error loading {name} ({series_id}): {e}")

monthly_data = monthly_data.sort_index()
monthly_data.to_csv("us_monthly_macro_data.csv")

working_hour_manuf loaded.
CPI loaded.
Ids_Prd loaded.
R_Csump loaded.
Ef_Fed_Rate loaded.
Intr_10Y loaded.
Unemp loaded.
M2 loaded.
M1 loaded.
Labor_partic loaded.
Per_savings loaded.
All_Emp loaded.
Moody_aaa loaded.
NetExports loaded.


In [ ]:
# Collect S&P 500 Index data from Yahoo Finance
# S&P500 100 for UK
# Ticker for S&P 500
ticker = "^GSPC"

start_date = "1992-01-01"
end_date = "2024-12-31"

# Download S&P 500 data (daily frequency)
data = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)

# Calculate monthly average of the 'Close' price
monthly_data = data['Close'].resample('M').mean()
monthly_data.name = "S&P500_MonthlyAvg"

monthly_data.to_csv("sp500_monthly_avg.csv")


/var/folders/1b/b3lsjhy10fsd753tw_292d5w0000gn/T/ipykernel_71070/3404505325.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = data['Close'].resample('M').mean()


In [ ]:
## Collect UK macroeconomic data from FRED API
uk_monthly_series = {
    'Share_Prices': 'SPASTT01GBM661N',       # Financial Market: Share Prices for United Kingdom
    'CPI': 'GBRCPIALLMINMEI',                # Consumer Price Index: Total for United Kingdom
    'Prod_Vol': 'GBRPROINDMISMEI',           # Production Volume: Industry (Except Construction) for United Kingdom
    'Intr_Rate': 'INTGSBGBM193N',            # Interest Rates, Government Securities, Government Bonds
    'Intr_10Y': 'IRLTLT01GBM156N',           # 10-Year Government Bond Yields for the United Kingdom
    'Total Reserves': 'TRESEGGBM052N',       # Total Reserves excluding Gold for the United Kingdom
    #'Uncertainty': 'UKEPUINDXM',            # Economic Policy Uncertainty Index for United Kingdom (Available from 1997)
    'Exchange': 'EXUSUK',                    # U.S. Dollars to U.K. Pound Sterling Spot Exchange Rate
    'Exports': 'XTEXVA01GBM664S',            # Exports: Commodities for United Kingdom
    'Imports': 'XTIMVA01GBM664S',            # Imports: Commodities for United Kingdom
}

start_date = datetime(1991, 1, 1)
end_date = datetime(2024, 12, 31)

monthly_data_uk = pd.DataFrame()

for name, series_id in uk_monthly_series.items():
    try:
        series = web.DataReader(series_id, 'fred', start_date, end_date)
        series = series.rename(columns={series_id: name})
        
        if monthly_data_uk.empty:
            monthly_data_uk = series
        else:
            monthly_data_uk = monthly_data_uk.join(series, how='outer')
        print(f"{name} loaded.")
    except Exception as e:
        print(f"Error loading {name} ({series_id}): {e}")

monthly_data_uk = monthly_data_uk.sort_index()
monthly_data_uk.to_csv("uk_monthly_macro_data.csv")

Share_Prices loaded.
CPI loaded.
Prod_Vol loaded.
Intr_Rate loaded.
Intr_10Y loaded.
Total Reserves loaded.
Uncertainty loaded.
Exchange loaded.
Exports loaded.
Imports loaded.


In [3]:
# Collect FTSE 100 Index data from Yahoo Finance
# FTSE 100 for UK
ticker = "^FTSE"

start_date = "1991-01-01"
end_date = "2024-12-31"

data = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)

monthly_data = data['Close'].resample('MS').mean() 
monthly_data.name = "FTSE100_MonthlyAvg"

monthly_data.to_csv("ftse100_monthly_avg.csv")

YF.download() has changed argument auto_adjust default to True


In [8]:
## Collect China macroeconomic data from FRED API

china_monthly_series = {
    'CPI': 'CPALTT01CNM657N',          # Consumer Price Index (1993.02 - 2024.03)
    'PPI': 'WPU1261',                  # Producer Price Index by Commodity: Furniture and Household Durables: Vitreous China, Fine Earthenware, and Other Pottery Products
    'Total_reserves': 'TRESEGCNM052N', # Total Reserves excluding Gold for China 
    'Exports': 'XTEXVA01CNM664S',      # Exports Value (Goods)
    'Imports': 'XTIMVA01CNM664S',      # Imports Value (Goods)
    'US imports': 'IMPCH',             # U.S. Imports of Goods by Customs Basis from China
    'Exchange': 'EXCHUS',              # Chinese Yuan Renminbi to U.S. Dollar Spot Exchange Rate
    'Eff. Exchange Rate': 'RBCNBIS',   # Real Broad Effective Exchange Rate for China
    #'Intr_Rate': 'IR3TTS01CNM156N',   # Interest Rates: 3-Month or 90-Day Rates and Yields: Treasury Securities: Total for China
    'Uncertainty': 'CHNMAINLANDEPU',   # Economic Policy Uncertainty Index: Mainland Papers for China
}

start_date = datetime(1994, 1, 1)
end_date = datetime(2023, 9, 30)

monthly_china_data = pd.DataFrame()

for name, series_id in china_monthly_series.items():
    try:
        series = web.DataReader(series_id, 'fred', start_date, end_date)
        series = series.rename(columns={series_id: name})
        if monthly_china_data.empty:
            monthly_china_data = series
        else:
            monthly_china_data = monthly_china_data.join(series, how='outer')
        print(f"{name} loaded.")
    except Exception as e:
        print(f"Error loading {name} ({series_id}): {e}")

monthly_china_data = monthly_china_data.sort_index()
monthly_china_data.to_csv("china_monthly_macro_data.csv")


CPI loaded.
PPI loaded.
Total_reserves loaded.
Exports loaded.
Imports loaded.
US imports loaded.
Exchange loaded.
Eff. Exchange Rate loaded.
Uncertainty loaded.


In [10]:
import pandas as pd
from pandas_datareader import data as web
from datetime import datetime

# Define FRED series
china_monthly_series = {
    'CLI': 'CHNLORSGPNOSTSAM',  # Composite Leading Indicator
}

start_date = datetime(1994, 1, 1)
end_date = datetime(2023, 9, 30)

# Load monthly data
monthly_china_data = pd.DataFrame()

for name, series_id in china_monthly_series.items():
    try:
        series = web.DataReader(series_id, 'fred', start_date, end_date)
        series = series.rename(columns={series_id: name})
        if monthly_china_data.empty:
            monthly_china_data = series
        else:
            monthly_china_data = monthly_china_data.join(series, how='outer')
        print(f"{name} loaded.")
    except Exception as e:
        print(f"Error loading {name} ({series_id}): {e}")

# Sort and save monthly data
monthly_china_data = monthly_china_data.sort_index()
monthly_china_data.to_csv("china_monthly_cli_data.csv")

# Create quarterly data =
# Resample to quarterly average
quarterly_avg = monthly_china_data.resample('Q').mean()

# Shift index to quarter's first month
quarterly_avg.index = quarterly_avg.index.to_period('Q').to_timestamp(how='start')

# Save to CSV
quarterly_avg.to_csv("china_quarterly_cli_data.csv")

print("Both monthly and quarterly CLI data saved.")


CLI loaded.
Both monthly and quarterly CLI data saved.


/var/folders/1b/b3lsjhy10fsd753tw_292d5w0000gn/T/ipykernel_83986/4081194625.py:34: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarterly_avg = monthly_china_data.resample('Q').mean()


In [2]:
## Collect Germany macroeconomic data from FRED API
germany_monthly_series = {
    'CPI': 'DEUCPIALLMINMEI',           # Consumer Price Index
    'Unemp': 'LRHUTTTTDEM156S',         # Unemployment Rate
    'Prod_Vol': 'DEUPROINDMISMEI',      # Production, Sales, Work Started and Orders: Production Volume: Economic Activity: Industry (Except Construction) for Germany
    'Retail': 'DEUSARTMISMEI',          # Retail Trade, Except of Motor Vehicles and Motorcycles for Germany
    'Exports': 'XTEXVA01DEM664S',       # Exports (Goods)
    'Imports': 'XTIMVA01DEM664S',       # Imports (Goods)
    'Manuf': 'DEUPRMNTO01GPSAM',        # Total Manufacturing for Germany
    'Total Reserves': 'TRESEGDEM052N',  # Total Reserves excluding Gold for Germany
    'Eff_Ex_Rate': 'RNDEBIS',           # Real Narrow Effective Exchange Rate for Germany
    'Uncertainty': 'EUEPUINDXM'         # Economic Policy Uncertainty Index for Europe (Germany, the United Kingdom, France, Italy, and Spain)
    
}

start_date = datetime(1991, 1, 1)
end_date = datetime(2024, 12, 31)

monthly_germany_data = pd.DataFrame()

for name, series_id in germany_monthly_series.items():
    try:
        series = web.DataReader(series_id, 'fred', start_date, end_date)
        series = series.rename(columns={series_id: name})
        if monthly_germany_data.empty:
            monthly_germany_data = series
        else:
            monthly_germany_data = monthly_germany_data.join(series, how='outer')
        print(f"{name} loaded.")
    except Exception as e:
        print(f"Error loading {name} ({series_id}): {e}")

monthly_germany_data = monthly_germany_data.sort_index()
monthly_germany_data.to_csv("germany_monthly_macro_data.csv")


CPI loaded.
Unemp loaded.
Prod_Vol loaded.
Retail loaded.
Exports loaded.
Imports loaded.
Manuf loaded.
Total Reserves loaded.
Eff_Ex_Rate loaded.
Uncertainty loaded.


In [ ]:
# Collect DAX Index data from Yahoo Finance
# DAX for Germany
ticker = "^GDAXI"

start_date = "1991-01-01"
end_date = "2024-12-31"

data = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)

monthly_data = data['Close'].resample('MS').mean() 
monthly_data.name = "DAX_MonthlyAvg"

monthly_data.to_csv("dax_monthly_avg.csv")

YF.download() has changed argument auto_adjust default to True
